In [53]:
from collections import OrderedDict
import pandas as pd
import numpy as np
import spacy
import re

DIR = 'OtherLists'
path = DIR + '/Got3.txt'
got = open(path).read()

# Author typo notice & fix:
print("Dialogs open/close marks: {}/{}".format(got.count('“'), got.count('”')))
where = 'And your hair, your golden hair...'
where_idx = got.index(where) + len(where)
got = got[:where_idx] +'”'+ got[where_idx:]
print("Dialogs open/close marks: {}/{}\n".format(got.count('“'), got.count('”')))

print(got[:300])

Dialogs open/close marks: 182/181
Dialogs open/close marks: 182/182

The king is dead, they told him, never knowing that Joffrey was his son as well as his
sovereign.
 “The Imp opened his throat with a dagger,” a costermonger declared at the roadside inn where
they spent the night. “He drank his blood from a big gold chalice.” The man did not recognize
the bearded on


In [54]:
# Initial cleaning
got = " ".join(got.split('\n'))
got = got.replace('  ', ' ')
got = got.replace('   ', ' ')
got = got.replace('...', '369')



def replace_text_with_mapped_tag(i, tag, dialogs, dialog, got):
    key = "D_"+str(i)+tag+"_D"
    dialogs[key] = dialog
    got = got.replace(dialog, key)
    return dialogs, got

punct_map = {"69":"_369", # <- 369, and last 2 digits of it stands for '...'
              ".":"_dot",   ",":"_coma",   "?":"_question",    "!":"_exc",    "-":"_dash",}


dialogs = OrderedDict(dict())
for i in range(1, got.count('“')):
    # Temporary replace dialogs with mapped tags
    open_quote, end_quote = got.index('“'), got.index('”')
    assert open_quote < end_quote
    
    used = False
    dialog = got[open_quote:end_quote + 1]
    last2_letters = dialog[-3:-1]
    for punct, tag in punct_map.items():
        if last2_letters.endswith(punct) or last2_letters[-1].isalpha():
            dialogs, got = replace_text_with_mapped_tag(i, tag, dialogs, dialog, got)
            used = True
    if not used:
        print("Other punct: _{}_ \nin: {})".format(last2, dialog))
got_s = [s.strip() for s in got.split(" ") if s]


def new_sent(sent, w, sents):
    sent += " " + w
    sents.append(sent)
    return "", sents

sents = []
sent = ""
for i, w in enumerate(got_s):
    # Glue lines back, word by word, according to new rules
    if w.endswith("!") or w.endswith("."):
        sent, sents = new_sent(sent, w, sents)
    elif w.endswith('?') or w.endswith("369"):
        if got_s[i+1].islower():
            sent += " " + w
        else:
            sent, sents = new_sent(sent, w, sents)
    elif w.startswith('D_') and w.endswith('_D'):
        cond_q = ('_question' in w and got_s[i+1].islower())
        if cond_q or 'coma' in w or ('_369' in w and got_s[i+1].islower()):
            sent += " " + w
        elif any(word in w for word in ['dash', '_dot', '_exc', '_alpha', '_369']):
            sent, sents = new_sent(sent, w, sents)
    else:
        sent += " " + w

remapped_got_lines = []
for sent in sents:
    # Revive dialogs back as separate or connected lines accodrding to rules planted in sents
    if 'D_' in sent:
        for w in sent.split(" "):
            if w.startswith("D_"):
                sent = sent.replace(w, dialogs[w])
    remapped_got_lines.append(sent)
        
remapped_got_lines = [line.replace('369', '...') for line in remapped_got_lines]

for i, sent in enumerate(remapped_got_lines):
    print(i, ">", sent)

0 >  The king is dead, they told him, never knowing that Joffrey was his son as well as his sovereign.
1 >  “The Imp opened his throat with a dagger,” a costermonger declared at the roadside inn where they spent the night.
2 >  “He drank his blood from a big gold chalice.”
3 >  The man did not recognize the bearded one-handed knight with the big bat on his shield, no more than any of them, so he said things he might otherwise have swallowed, had he known who was listening.
4 >  “It was poison did the deed,” the innkeep insisted.
5 >  “The boy’s face turned black as a plum.”
6 >  “May the Father judge him justly,” murmured a septon.
7 >  “The dwarf’s wife did the murder with him,” swore an archer in Lord Rowan’s livery.
8 >  “Afterward, she vanished from the hall in a puff of brimstone, and a ghostly direwolf was seen prowling the Red Keep, blood dripping from his jaws.”
9 >  Jaime sat silent through it all, letting the words wash over him, a horn of ale forgotten in his one good hand.


In [16]:
dialogs

OrderedDict([('D_1_coma_D', '“The Imp opened his throat with a dagger,”'),
             ('D_2_dot_D', '“He drank his blood from a big gold chalice.”'),
             ('D_3_coma_D', '“It was poison did the deed,”'),
             ('D_4_dot_D', '“The boy’s face turned black as a plum.”'),
             ('D_5_coma_D', '“May the Father judge him justly,”'),
             ('D_6_coma_D', '“The dwarf’s wife did the murder with him,”'),
             ('D_7_dot_D',
              '“Afterward, she vanished from the hall in a puff of brimstone, and a ghostly direwolf was seen prowling the Red Keep, blood dripping from his jaws.”'),
             ('D_8_question_D', '“What’s that awful stink?”'),
             ('D_9_question_D',
              '“Smoke, sweat, and shit. King’s Landing, in short. If you have a good nose you can smell the treachery too. You’ve never smelled a city before?”'),
             ('D_10_dot_D',
              '“I smelled White Harbor. It never stank like this.”'),
             ('D_11_d

In [17]:
got

'The king is dead, they told him, never knowing that Joffrey was his son as well as his sovereign. D_1_coma_D a costermonger declared at the roadside inn where they spent the night. D_2_dot_D The man did not recognize the bearded one-handed knight with the big bat on his shield, no more than any of them, so he said things he might otherwise have swallowed, had he known who was listening. D_3_coma_D the innkeep insisted. D_4_dot_D D_5_coma_D murmured a septon. D_6_coma_D swore an archer in Lord Rowan’s livery. D_7_dot_D Jaime sat silent through it all, letting the words wash over him, a horn of ale forgotten in his one good hand. Joffrey. My blood. My firstborn. My son. He tried to bring the boy’s face to mind, but his features kept turning into Cersei’s. She will be in mourning, her hair in disarray and her eyes red from crying, her mouth trembling as she tries to speak. She will cry again when she sees me, though she’ll fight the tears. His sister seldom wept but when she was with him

In [18]:
got_s = [s.strip() for s in got.split(" ") if s]
for i, s in enumerate(got_s):
    print(i, s)

0 The
1 king
2 is
3 dead,
4 they
5 told
6 him,
7 never
8 knowing
9 that
10 Joffrey
11 was
12 his
13 son
14 as
15 well
16 as
17 his
18 sovereign.
19 D_1_coma_D
20 a
21 costermonger
22 declared
23 at
24 the
25 roadside
26 inn
27 where
28 they
29 spent
30 the
31 night.
32 D_2_dot_D
33 The
34 man
35 did
36 not
37 recognize
38 the
39 bearded
40 one-handed
41 knight
42 with
43 the
44 big
45 bat
46 on
47 his
48 shield,
49 no
50 more
51 than
52 any
53 of
54 them,
55 so
56 he
57 said
58 things
59 he
60 might
61 otherwise
62 have
63 swallowed,
64 had
65 he
66 known
67 who
68 was
69 listening.
70 D_3_coma_D
71 the
72 innkeep
73 insisted.
74 D_4_dot_D
75 D_5_coma_D
76 murmured
77 a
78 septon.
79 D_6_coma_D
80 swore
81 an
82 archer
83 in
84 Lord
85 Rowan’s
86 livery.
87 D_7_dot_D
88 Jaime
89 sat
90 silent
91 through
92 it
93 all,
94 letting
95 the
96 words
97 wash
98 over
99 him,
100 a
101 horn
102 of
103 ale
104 forgotten
105 in
106 his
107 one
108 good
109 hand.
110 Joffrey.
111 My
112 blood.
113

935 of
936 such
937 a
938 companion.
939 D_27_369_D
940 D_28_dot_D
941 Then
942 why
943 did
944 you
945 ever
946 bother
947 putting
948 on
949 a
950 sword?
951 he
952 might
953 have
954 said,
955 but
956 he
957 bit
958 back
959 the
960 words.
961 D_29_dot_D
962 One-handed,
963 he
964 wheeled
965 his
966 horse
967 about
968 and
969 left
970 her.
971 The
972 Gate
973 of
974 the
975 Gods
976 was
977 open
978 when
979 they
980 reached
981 it,
982 but
983 two
984 dozen
985 wayns
986 were
987 lined
988 up
989 along
990 the
991 roadside,
992 loaded
993 with
994 casks
995 of
996 cider,
997 barrels
998 of
999 apples,
1000 bales
1001 of
1002 hay,
1003 and
1004 some
1005 of
1006 the
1007 biggest
1008 pumpkins
1009 Jaime
1010 had
1011 ever
1012 seen.
1013 Almost
1014 every
1015 wagon
1016 had
1017 its
1018 guards;
1019 men-at-arms
1020 wearing
1021 the
1022 badges
1023 of
1024 small
1025 lordlings,
1026 sellswords
1027 in
1028 mail
1029 and
1030 boiled
1031 leather,
1032 sometimes
1033 only
1034 a

2435 step.
2436 D_130_dot_D
2437 D_131_dot_D
2438 D_132_coma_D
2439 she
2440 repeated,
2441 turning
2442 away.
2443 Jaime
2444 laced
2445 up
2446 his
2447 breeches
2448 and
2449 did
2450 as
2451 she
2452 commanded.
2453 Weary
2454 as
2455 he
2456 was,
2457 he
2458 could
2459 not
2460 seek
2461 a
2462 bed.
2463 By
2464 now
2465 his
2466 lord
2467 father
2468 knew
2469 that
2470 he
2471 was
2472 back
2473 in
2474 the
2475 city.
2476 The
2477 Tower
2478 of
2479 the
2480 Hand
2481 was
2482 guarded
2483 by
2484 Lannister
2485 household
2486 guards,
2487 who
2488 knew
2489 him
2490 at
2491 once.
2492 D_133_coma_D
2493 one
2494 said,
2495 as
2496 he
2497 held
2498 the
2499 door.
2500 D_134_dot_D
2501 He
2502 climbed
2503 the
2504 stairs
2505 and
2506 pushed
2507 into
2508 the
2509 solar
2510 unannounced,
2511 to
2512 find
2513 his
2514 father
2515 sitting
2516 by
2517 the
2518 fire.
2519 Lord
2520 Tywin
2521 was
2522 alone,
2523 for
2524 which
2525 Jaime
2526 was
2527 thankful.
2528 He
2529 h